# SEMAS-COMPLETE: FIXED METRICS - ADAPTIVE THRESHOLD & PROPER BEARING LABELS
## Issue: Bearing dataset returns continuous RUL values, not binary labels (0/1)
## Fix: Convert RUL ≤ 25 to anomaly (1), RUL > 25 to normal (0)

In [ ]:
 %pip install langchain_openai langchain langchain-community vinagent==0.0.6 numpy pandas scikit-learn tensorflow langgraph scipy matplotlib seaborn opendatasets spektral paho-mqtt shap -q

# from google.colab import drive
# drive.mount('/content/drive')

import pandas as pd
import numpy as np
import json
import gc
import os
import warnings
import threading
import time
import queue
from datetime import datetime
from collections import defaultdict, deque
warnings.filterwarnings('ignore')

from typing import Dict, Any, List, Tuple

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, mean_absolute_error, mean_squared_error
from sklearn.metrics import confusion_matrix

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras.layers import Dense, RepeatVector, TimeDistributed, Input, Dropout, MultiHeadAttention, LayerNormalization, Conv1D, Add, GlobalAveragePooling1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from langchain_community.llms import OpenAI  # ✅ correct

from vinagent.multi_agent import AgentNode
import matplotlib.pyplot as plt
import seaborn as sns
import langchain
langchain.debug = False

import paho.mqtt.client as mqtt
from abc import ABC, abstractmethod

import shap

# NEW: Import for LLM integration
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# NEW: Import for statistical testing (drift detection)
from scipy.stats import ks_2samp

print("✅ All imports successful including LLM, RUL, and Feedback Loop support")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 19.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.4/441.4 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [ ]:
MQTT_CONFIG = {
    'broker': 'localhost',
    'port': 1883,
    'keepalive': 60,
    'qos': 1,
    'tls_version': None,
    'use_tls': False,
    'topics': {
        'edge_stream_1': 'chunk/stream1',
        'edge_stream_2': 'chunk/stream2',
        'if_scores': 'scores/if',
        'transformer_scores': 'scores/transformer',
        'rul_predictions': 'predictions/rul',
        'anomalies': 'anomalies',
        'actions': 'actions',
        'local_feedback': 'feedback/edge',
        'operator_feedback': 'feedback/operator',
        'feedback_metrics': 'feedback/metrics',
        'policy_updates': 'policy/updates',
        'knowledge_graph': 'graph/sync',
        'monitoring_logs': 'monitor/logs',
        'evaluation_metrics': 'metrics/evaluation'
    }   
}

class MessageBroker:
    def __init__(self):
        self.topics = defaultdict(deque)
        self.subscribers = defaultdict(list)
        self.lock = threading.Lock()
        self.message_history = defaultdict(list)

    def publish(self, topic: str, payload: Dict[str, Any], qos: int = 1):
        with self.lock:
            message = {'timestamp': datetime.now().isoformat(), 'topic': topic, 'payload': payload, 'qos': qos}
            self.topics[topic].append(message)
            self.message_history[topic].append(message)
            for callback in self.subscribers[topic]:
                try:
                    callback(topic, message)
                except Exception as e:
                    pass

    def subscribe(self, topic: str, callback):
        with self.lock:
            self.subscribers[topic].append(callback)

    def get_latest(self, topic: str, default=None):
        with self.lock:
            if self.topics[topic]:
                return self.topics[topic][-1]
            return default

    def get_all(self, topic: str):
        with self.lock:
            return list(self.topics[topic])

message_broker = MessageBroker()
print("✅ Message Broker initialized")

✅ Message Broker initialized


In [ ]:
from collections import Counter

print("\n📂 Loading Boiler Dataset...")
data_boiler = pd.read_csv('dataset/Boiler_emulator_dataset.csv')
print(f"✅ Boiler loaded: {data_boiler.shape}")

print("\n📂 Loading Wind Turbine Dataset...")
try:
    scada_df = pd.read_csv('dataset/iiot-data-of-wind-turbine/scada_data.csv')
    scada_df['DateTime'] = pd.to_datetime(scada_df['DateTime'])

    fault_df = pd.read_csv('dataset/iiot-data-of-wind-turbine/fault_data.csv')
    fault_df['DateTime'] = pd.to_datetime(fault_df['DateTime'])

    status_df = pd.read_csv('dataset/iiot-data-of-wind-turbine/status_data.csv')
    status_df['Time'] = pd.to_datetime(status_df['Time'])
    status_df.rename(columns={'Time': 'DateTime'}, inplace=True)

    print("✅ Wind Turbine dataset loaded from dataset/iiot-data-of-wind-turbine/")
    data_source = "Real dataset"

except FileNotFoundError as e:
    print(f"⚠️  Dataset not found: {e}")
    print("📝 To fix this: Download the wind turbine CSV files and place them in:")
    print("   dataset/iiot-data-of-wind-turbine/")
    print("   ├── scada_data.csv")
    print("   ├── fault_data.csv")
    print("   └── status_data.csv")
    print("\n💾 Generating synthetic wind turbine data as fallback...\n")

    n_samples = 1000
    n_features = 40
    np.random.seed(42)
    scada_data = {'DateTime': pd.date_range('2014-04-01', periods=n_samples, freq='10min')}
    for i in range(n_features):
        scada_data[f'Sensor_{i}'] = np.random.normal(50, 15, n_samples)

    scada_df = pd.DataFrame(scada_data)
    scada_df['DateTime'] = pd.to_datetime(scada_df['DateTime'])

    fault_types = ['gf', 'mf', 'ff', 'af', 'ef']
    fault_data = {
        'DateTime': scada_df['DateTime'].sample(n=200, random_state=42).sort_values().reset_index(drop=True),
        'Fault': np.random.choice(fault_types, size=200)
    }
    fault_df = pd.DataFrame(fault_data)

    status_df = pd.DataFrame({
        'DateTime': scada_df['DateTime'],
        'Status': np.random.choice(['OK', 'WARNING', 'ERROR'], size=n_samples, p=[0.7, 0.2, 0.1])
    })

    print("✅ Synthetic wind turbine data generated")
    data_source = "Synthetic (fallback)"

df_turbine = scada_df.merge(fault_df[['DateTime', 'Fault']], on='DateTime', how='left')
df_turbine['Fault'] = df_turbine['Fault'].replace(np.nan, 'NF')

df_nf = df_turbine[df_turbine['Fault']=='NF']
if len(df_nf) > 300:
    df_nf = df_nf.sample(n=300, random_state=42)

df_f = df_turbine[df_turbine['Fault']!='NF']
df_turbine = pd.concat((df_nf, df_f), axis=0).reset_index(drop=True)

irrelevant_cols = ['DateTime', 'WEC: ava. windspeed', 'WEC: ava. available P from wind',
                   'WEC: ava. available P technical reasons', 'WEC: ava. Available P force majeure reasons',
                   'WEC: ava. Available P force external reasons', 'WEC: max. windspeed', 'WEC: min. windspeed',
                   'WEC: Operating Hours', 'WEC: Production kWh', 'WEC: Production minutes']
irrelevant_cols = [col for col in irrelevant_cols if col in df_turbine.columns]
df_turbine_clean = df_turbine.drop(columns=irrelevant_cols, errors='ignore')

X_turbine_full = df_turbine_clean.drop('Fault', axis=1)
X_turbine_full['sensor_diff_0_1'] = X_turbine_full.iloc[:, 0] - X_turbine_full.iloc[:, 1]
X_turbine_full['sensor_change_0'] = X_turbine_full.iloc[:, 0].diff().fillna(0)
y_turbine_full = df_turbine_clean['Fault']

le_turbine = LabelEncoder()
y_turbine_encoded = le_turbine.fit_transform(y_turbine_full)

X_train_turbine, X_test_turbine, y_train_turbine, y_test_turbine = train_test_split(
    X_turbine_full, y_turbine_encoded, test_size=0.2, random_state=42
)

scaler_wt = StandardScaler()
X_train_turbine_scaled = pd.DataFrame(scaler_wt.fit_transform(X_train_turbine), columns=X_train_turbine.columns)
X_test_turbine_scaled = pd.DataFrame(scaler_wt.transform(X_test_turbine), columns=X_test_turbine.columns)

df_clean = data_boiler.copy()
le_condition = LabelEncoder()
df_clean['Condition'] = le_condition.fit_transform(df_clean['Condition'])
le_class = LabelEncoder()
df_clean['Class'] = le_class.fit_transform(df_clean['Class'])
numeric_cols = ['Fuel_Mdot', 'Tair', 'Treturn', 'Tsupply', 'Water_Mdot']
for col in numeric_cols:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
df_clean = df_clean.dropna()
X_boiler = df_clean.drop('Class', axis=1)
y_boiler = df_clean['Class']

# Add domain-specific features BEFORE splitting
#X_boiler['temp_diff'] = X_boiler['Tsupply'] - X_boiler['Treturn']
#X_boiler['temp_change'] = X_boiler['Tsupply'].diff().fillna(0)

# Add rolling statistics (3-point windows)
#for col in ['Tsupply', 'Treturn', 'FuelMdot']:
 #   X_boiler[f'{col}_rolling_mean'] = X_boiler[col].rolling(window=3, min_periods=1).mean()
#    X_boiler[f'{col}_rolling_std'] = X_boiler[col].rolling(window=3, min_periods=1).std().fillna(0)


X_train_boiler_full, X_test_boiler, y_train_boiler_full, y_test_boiler = train_test_split(
    X_boiler, y_boiler, test_size=0.2, random_state=42, stratify=y_boiler
)

scaler_b = StandardScaler()
X_train_boiler = pd.DataFrame(scaler_b.fit_transform(X_train_boiler_full), columns=X_train_boiler_full.columns)
X_test_boiler = pd.DataFrame(scaler_b.transform(X_test_boiler), columns=X_test_boiler.columns)


print(f"✅ Boiler - Train {X_train_boiler.shape}, Test {X_test_boiler.shape}")
print(f"✅ Wind Turbine - Train {X_train_turbine_scaled.shape}, Test {X_test_turbine_scaled.shape}")

y_test_boiler_anomaly = (y_test_boiler == 1).astype(int)
y_test_turbine_anomaly = (y_test_turbine > 0).astype(int)

y_test_boiler_rul = pd.Series(
    np.where(y_test_boiler == 1, np.random.randint(5, 30, len(y_test_boiler)), np.random.randint(30, 100, len(y_test_boiler)))
)
y_test_turbine_rul = pd.Series(
    np.where(y_test_turbine_anomaly == 1, np.random.randint(5, 25, len(y_test_turbine)), np.random.randint(25, 100, len(y_test_turbine)))
)

print(f"\n✅ Data loading complete")
print(f"   Boiler anomaly class distribution: {y_test_boiler_anomaly.value_counts().to_dict()}")
print(f"   Wind Turbine anomaly class distribution: {dict(Counter(y_test_turbine_anomaly))}")
print(f"   Wind Turbine RUL threshold: 25 hours (RUL <= 25 = anomaly)")
print(f"   Data source: {data_source}")

Xtr_boiler, Xte_boiler = X_train_boiler, X_test_boiler
yte_boiler_anom, yte_boiler_rul = y_test_boiler_anomaly.values, y_test_boiler_rul.values
Xtr_turbine, Xte_turbine = X_train_turbine_scaled, X_test_turbine_scaled
yte_turbine_anom, yte_turbine_rul = y_test_turbine_anomaly, y_test_turbine_rul.values



📂 Loading Boiler Dataset...
✅ Boiler loaded: (27280, 7)

📂 Loading Wind Turbine Dataset...
✅ Wind Turbine dataset loaded from dataset/iiot-data-of-wind-turbine/
✅ Boiler - Train (21824, 6), Test (5456, 6)
✅ Wind Turbine - Train (263, 57), Test (66, 57)

✅ Data loading complete
   Boiler anomaly class distribution: {0: 3696, 1: 1760}
   Wind Turbine anomaly class distribution: {np.int64(1): 61, np.int64(0): 5}
   Wind Turbine RUL threshold: 25 hours (RUL <= 25 = anomaly)
   Data source: Real dataset


In [ ]:
CONFIG = {
    'datasets': {
        'boiler': {
            'X_train': X_train_boiler,
            'X_test': X_test_boiler,
            'y_train': y_train_boiler_full,
            'y_test': y_test_boiler,
            'y_test_anomaly': y_test_boiler_anomaly,
            'y_test_rul': y_test_boiler_rul,
            'scaler': scaler_b,
            'name': 'Boiler Emulator',
        },
        'wind_turbine': {
            'X_train': X_train_turbine_scaled,
            'X_test': X_test_turbine_scaled,
            'y_train': y_train_turbine,
            'y_test': y_test_turbine,
            'y_test_anomaly': y_test_turbine_anomaly,
            'y_test_rul': y_test_turbine_rul,
            'scaler': scaler_wt,
            'name': 'Wind Turbine',
        },
    },
    'mqtt_broker': message_broker,
}

class MQTTAgent(ABC):
    def __init__(self, name: str, broker: 'MessageBroker', subscribed_topics: List[str] = None):
        self.name = name
        self.broker = broker
        self.subscribed_topics = subscribed_topics or []
        self.message_queue = queue.Queue()
        self.execution_log = []
        for topic in self.subscribed_topics:
            self.broker.subscribe(topic, self.message_handler)

    def message_handler(self, topic: str, message: Dict[str, Any]):
        self.message_queue.put({'topic': topic, 'message': message})

    def publish(self, topic: str, payload: Dict[str, Any], qos: int = None):
        qos = qos or MQTT_CONFIG['qos']
        self.broker.publish(topic, payload, qos=qos)
        self.execution_log.append({'timestamp': datetime.now().isoformat(), 'action': 'publish', 'topic': topic, 'agent': self.name})

    def get_latest_message(self, topic: str):
        return self.broker.get_latest(topic)

    @abstractmethod
    def execute(self, *args, **kwargs):
        pass

print("✅ Configuration & Base Classes initialized")


✅ Configuration & Base Classes initialized


In [ ]:
class EdgeSubagentA1(MQTTAgent):
    def __init__(self, name='A1_Temperature', broker=None, window_size=10):
        super().__init__(name, broker or message_broker)
        self.window_size = window_size
        self.publishing_topic = MQTT_CONFIG['topics']['edge_stream_1']

    def compute_features(self, data: np.ndarray) -> Dict[str, np.ndarray]:
        rms = np.sqrt(np.mean(data**2, axis=0))
        kurtosis = np.mean((data - np.mean(data, axis=0))**4, axis=0) / (np.std(data, axis=0)**4 + 1e-8)
        skewness = np.mean((data - np.mean(data, axis=0))**3, axis=0) / (np.std(data, axis=0)**3 + 1e-8)
        x = np.arange(len(data))
        trend = np.array([np.polyfit(x, data[:, i], 1)[0] for i in range(data.shape[1])])
        fft_vals = np.fft.fft(data, axis=0)
        spectral_energy = np.abs(fft_vals).mean(axis=0)
        stats = {'mean': np.mean(data, axis=0), 'std': np.std(data, axis=0)}
        return {'rms': rms, 'kurtosis': kurtosis, 'skewness': skewness, 'trend': trend, 'spectral_energy': spectral_energy, 'stats': stats}

    def execute(self, data: np.ndarray, timestamp: float = None):
        timestamp = timestamp or datetime.now().timestamp()
        features = self.compute_features(data)
        payload = {
            'timestamp': timestamp,
            'agent': self.name,
            'window_size': self.window_size,
            'data_shape': data.shape,
            'features': {'rms': features['rms'].tolist(), 'kurtosis': features['kurtosis'].tolist()},
        }
        self.publish(self.publishing_topic, payload)
        return features

class EdgeSubagentA2(EdgeSubagentA1):
    def __init__(self, name='A2_Vibration', broker=None, window_size=10):
        super().__init__(name, broker, window_size)
        self.publishing_topic = MQTT_CONFIG['topics']['edge_stream_2']

print("✅ Edge Layer Agents (A1, A2) defined")

✅ Edge Layer Agents (A1, A2) defined


In [ ]:
class FogOrchestrationAgentB(MQTTAgent):
    def __init__(self, name='B_Orchestrator', broker=None):
        subscribed_topics = [MQTT_CONFIG['topics']['edge_stream_1'], MQTT_CONFIG['topics']['edge_stream_2']]
        super().__init__(name, broker or message_broker, subscribed_topics)
        self.score_history = deque(maxlen=100)

    def detect_drift(self, anomaly_scores: np.ndarray, window_size: int = 50) -> bool:
        if len(anomaly_scores) > 0:
            self.score_history.extend(anomaly_scores)
        if len(self.score_history) < window_size:
            return False
        recent = list(self.score_history)[-window_size:]
        historical = list(self.score_history)[:-window_size]
        if len(historical) < 10:
            return False
        statistic, p_value = ks_2samp(historical, recent)
        if p_value < 0.05:
            self.publish(MQTT_CONFIG['topics']['local_feedback'], {'drift_detected': True, 'p_value': float(p_value)})
            return True
        return False

    def execute(self, X: np.ndarray = None) -> Dict[str, Any]:
        msg_a1 = self.get_latest_message(MQTT_CONFIG['topics']['edge_stream_1'])
        msg_a2 = self.get_latest_message(MQTT_CONFIG['topics']['edge_stream_2'])
        if msg_a1 and msg_a2:
            payload = {
                'timestamp': datetime.now().isoformat(),
                'agent': self.name,
                'a1_status': 'received',
                'a2_status': 'received'
            }
            self.publish(MQTT_CONFIG['topics']['monitoring_logs'], payload)
            return {'status': 'orchestration_complete', 'agents_active': 2}
        return {'status': 'waiting_for_data', 'agents_active': 0}

print("✅ Fog Orchestration Agent defined")

✅ Fog Orchestration Agent defined


In [ ]:
class FogSubagentB1(MQTTAgent):
    def __init__(self, name='B1_IsolationForest', broker=None, contamination=0.32):
        super().__init__(name, broker or message_broker)
        self.contamination = contamination
        self.model = IsolationForest(contamination=self.contamination, random_state=42, n_estimators=200, max_samples=256)
        self.is_trained = False
        self.X_train = None

    def train(self, X_train):
        self.X_train = X_train  # Store training data for retraining
        self.model.fit(X_train)
        self.is_trained = True
        print(f'   ✅ B1 trained with contamination={self.contamination:.4f}')

    def update_contamination(self, new_contamination: float):
        """Update contamination parameter and retrain model"""
        if abs(new_contamination - self.contamination) > 0.01:  # Only retrain if significant change
            print(f'   🔄 B1 Retraining: contamination {self.contamination:.4f} → {new_contamination:.4f}')
            self.contamination = new_contamination
            self.model = IsolationForest(contamination=self.contamination, random_state=42, n_estimators=200, max_samples=256)
            if self.X_train is not None:
                self.model.fit(self.X_train)
                self.is_trained = True

    def execute(self, X: np.ndarray) -> Dict[str, Any]:
        if not self.is_trained:
            return None
        predictions = self.model.predict(X)
        scores = self.model.score_samples(X)
        payload = {'timestamp': datetime.now().isoformat(), 'agent': self.name, 'predictions': predictions.tolist(), 'anomaly_scores': scores.tolist()}
        self.publish(MQTT_CONFIG['topics']['if_scores'], payload)
        return {'predictions': predictions, 'scores': scores}

class FogSubagentB2(MQTTAgent):
    def __init__(self, name='B2_TimeSeriesTransformer', broker=None):
        super().__init__(name, broker or message_broker)
        self.model = None
        self.is_trained = False

    def build_transformer_model(self, input_dim: int, seq_length: int = 10, d_model: int = 128):
        inputs = Input(shape=(seq_length, input_dim))
        x = Conv1D(d_model, kernel_size=1, padding='same', activation='relu')(inputs)
        attention = MultiHeadAttention(num_heads=4, key_dim=16)
        attn_output = attention(x, x)
        x = Add()([x, attn_output])
        x = LayerNormalization()(x)
        x = Dropout(0.2)(x)
        x = GlobalAveragePooling1D()(x)
        x = Dense(32, activation='relu')(x)
        outputs = Dense(1, activation='sigmoid')(x)
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy')
        self.model = model

    def train(self, X_train):
        input_dim = X_train.shape[1]
        self.build_transformer_model(input_dim)
        X_seq = self.create_sequences(X_train, seq_length=10)
        y_train = np.zeros(len(X_seq))
        self.model.fit(X_seq, y_train, epochs=25, batch_size=16, verbose=0, callbacks=[EarlyStopping(monitor='loss', patience=2)])
        self.is_trained = True

    def execute(self, X: np.ndarray) -> Dict[str, Any]:
        if not self.is_trained:
            return None
        X_seq = self.create_sequences(X, seq_length=10)
        scores = self.model.predict(X_seq, verbose=0).flatten()
        scores = np.pad(scores, (9, len(X) - len(scores) - 9), mode='edge')
        payload = {'timestamp': datetime.now().isoformat(), 'agent': self.name, 'anomaly_scores': scores.tolist()}
        self.publish(MQTT_CONFIG['topics']['transformer_scores'], payload)
        return {'scores': scores}

    def create_sequences(self, data, seq_length=10):
        sequences = []
        for i in range(len(data) - seq_length + 1):
            sequences.append(data[i:i+seq_length])
        return np.array(sequences) if sequences else np.array([])

print("✅ Fog Detection Agents B1, B2 defined")

✅ Fog Detection Agents B1, B2 defined


In [ ]:
class FogSubagentB2_RUL(MQTTAgent):
    def __init__(self, name='B2_RUL_Predictor', broker=None):
        super().__init__(name, broker or message_broker)
        self.model = None
        self.is_trained = False

    def build_rul_model(self, input_dim: int):
        from tensorflow.keras.layers import LSTM
        inputs = Input(shape=(10, input_dim))
        x = LSTM(64, return_sequences=True)(inputs)
        x = LSTM(32)(x)
        x = Dense(32, activation='relu')(x)
        outputs = Dense(1, activation='relu')(x)
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mae')
        self.model = model

    def train(self, X_train, RUL_train):
        input_dim = X_train.shape[1]
        self.build_rul_model(input_dim)
        X_seq = self.create_sequences(X_train, seq_length=10)
        if len(X_seq) == 0:
            self.is_trained = True
            return
        RUL_train_array = RUL_train.values if hasattr(RUL_train, 'values') else RUL_train
        if len(RUL_train_array) < len(X_train):
            RUL_train_array = np.pad(RUL_train_array, (0, len(X_train) - len(RUL_train_array)), mode='edge')
        RUL_seq = RUL_train_array[9:9+len(X_seq)]
        if len(RUL_seq) != len(X_seq):
            if len(RUL_seq) < len(X_seq):
                RUL_seq = np.pad(RUL_seq, (0, len(X_seq) - len(RUL_seq)), mode='edge')
            else:
                RUL_seq = RUL_seq[:len(X_seq)]
        self.model.fit(X_seq, RUL_seq, epochs=10, batch_size=32, verbose=0, callbacks=[EarlyStopping(monitor='loss', patience=2)])
        self.is_trained = True

    def execute(self, X: np.ndarray) -> Dict[str, Any]:
        if not self.is_trained:
            return None
        X_seq = self.create_sequences(X, seq_length=10)
        if len(X_seq) == 0:
            return None
        rul_predictions = self.model.predict(X_seq, verbose=0).flatten()
        rul_predictions = np.pad(rul_predictions, (9, len(X) - len(rul_predictions) - 9), mode='edge')
        payload = {'timestamp': datetime.now().isoformat(), 'agent': self.name, 'rul_predictions': rul_predictions.tolist()}
        self.publish(MQTT_CONFIG['topics']['rul_predictions'], payload)
        return {'rul_predictions': rul_predictions}

    def create_sequences(self, data, seq_length=10):
        sequences = []
        for i in range(len(data) - seq_length + 1):
            sequences.append(data[i:i+seq_length])
        return np.array(sequences) if sequences else np.array([])

print("✅ RUL Prediction Agent defined")

✅ RUL Prediction Agent defined


In [ ]:
class FogSubagentB3(MQTTAgent):
    def __init__(self, name='B3_ConsensusVoting', broker=None):
        super().__init__(name, broker or message_broker)

    def aggregate(self) -> Dict[str, Any]:
        msg_b1 = self.get_latest_message(MQTT_CONFIG['topics']['if_scores'])
        msg_b2 = self.get_latest_message(MQTT_CONFIG['topics']['transformer_scores'])
        weights = self.get_latest_message(MQTT_CONFIG['topics']['knowledge_graph'])
        print(f'weights {weights}')
        if weights:
            w1 = weights.get('payload').get('w1', 0.4)
        else:
            w1 = 0.4
        w2 = 1-w1
        if msg_b1 and msg_b2:
            scores_b1 = np.array(msg_b1['payload']['anomaly_scores'])
            scores_b2 = np.array(msg_b2['payload']['anomaly_scores'])
            scores_b1_norm = (scores_b1 - scores_b1.min()) / (scores_b1.max() - scores_b1.min() + 1e-8)
            scores_b2_norm = (scores_b2 - scores_b2.min()) / (scores_b2.max() - scores_b2.min() + 1e-8)
            msg_b1 = self.get_latest_message(MQTT_CONFIG['topics']['if_scores'])
            msg_b1 = self.get_latest_message(MQTT_CONFIG['topics']['if_scores'])
            consensus_scores = w1 * scores_b1_norm + w2 * scores_b2_norm
            print(f"w1: {w1}; w2: {w2}")
            print(f'top 5 consensus_scores: {consensus_scores[:5]}')
            # FIX: Return normalized scores, not raw scores
            return {'consensus_scores': consensus_scores, 'scores_b1_norm': scores_b1_norm, 'scores_b2_norm': scores_b2_norm}
        return None

    def execute(self) -> Dict[str, Any]:
        result = self.aggregate()
        if result:
            payload = {
                'timestamp': datetime.now().isoformat(),
                'agent': self.name,
                'consensus_scores': result['consensus_scores'].tolist(),
                'scores_b1_norm': result['scores_b1_norm'].tolist(),
                'scores_b2_norm': result['scores_b2_norm'].tolist()
            }
            self.publish(MQTT_CONFIG['topics']['anomalies'], payload)
            return result
        return None

class FogSubagentC(MQTTAgent):
    def __init__(self, name='C_ResponseGenerator', broker=None, use_mock=True):
        super().__init__(name, broker or message_broker)
        self.use_mock = use_mock

    def generate_explanation_llm(self, anomaly_scores: np.ndarray) -> Dict[str, Any]:
        severity = float(np.max(anomaly_scores))
        if severity > 0.7:
            return {'explanation': 'Critical bearing wear detected', 'action': 'IMMEDIATE_SHUTDOWN', 'priority': 'HIGH', 'downtime': 8}
        elif severity > 0.4:
            return {'explanation': 'Moderate anomaly', 'action': 'SCHEDULE_INSPECTION', 'priority': 'MEDIUM', 'downtime': 4}
        else:
            return {'explanation': 'Minor drift', 'action': 'CONTINUE_MONITORING', 'priority': 'LOW', 'downtime': 0}

    def execute(self) -> Dict[str, Any]:
        anomaly_msg = self.get_latest_message(MQTT_CONFIG['topics']['anomalies'])
        if anomaly_msg:
            scores = np.array(anomaly_msg['payload']['consensus_scores'])
            explanation_dict = self.generate_explanation_llm(scores)
            payload = {'timestamp': datetime.now().isoformat(), 'agent': self.name, **explanation_dict}
            self.publish(MQTT_CONFIG['topics']['actions'], payload)
            return explanation_dict
        return None

print("✅ Fog Detection Agents B3, C defined")

✅ Fog Detection Agents B3, C defined


In [ ]:
class CloudAgentD(MQTTAgent):
    def __init__(self, name='D_EvolutionAgent_PPO', broker=None):
        super().__init__(name, broker or message_broker)
        self.policy_params = {
            'contamination_rate': 0.32,
            'threshold_delta_consensus': 0.0,  # Store DELTAS not absolute values
            'threshold_delta_b1': 0.0,
            'threshold_delta_b2': 0.0,
            'w1': 0.4,
            'w2': 0.6
        }
        self.data_quantity = 0
        self.learning_rates = {
            'w1': 0.1,
            'contamination': 0.02,
            'threshold': 0.05
        }

    def update_policy_ppo(self, metrics: Dict) -> Dict:
        f1_consensus = metrics.get('consensus').get('f1', 0.5)
        f1_b1_score = metrics.get('scores_b1_norm').get('f1', 0.2)
        f1_b2_score = metrics.get('scores_b2_norm').get('f1', 0.2)

        # Update w1/w2 weights
        delta_w1 = self.learning_rates['w1'] * (f1_b1_score - f1_b2_score)

        weights = self.get_latest_message(MQTT_CONFIG['topics']['knowledge_graph'])
        if weights:
            w1_current = weights.get('payload').get('w1', 0.4)
            contamination_current = weights.get('payload').get('contamination_rate', 0.32)
        else:
            w1_current = self.policy_params['w1']
            contamination_current = self.policy_params['contamination_rate']

        epsilon = 0.005
        w1_new = np.clip(w1_current + delta_w1, 0.2, 0.6) + epsilon
        w2_new = 1.0 - w1_new

        # Tune contamination based on F1 performance (increase if F1 < 0.6, decrease if F1 > 0.7)
        if f1_consensus < 0.6:
            delta_contamination = self.learning_rates['contamination']
        elif f1_consensus > 0.7:
            delta_contamination = -self.learning_rates['contamination']
        else:
            delta_contamination = 0.0
        contamination_new = np.clip(contamination_current + delta_contamination, 0.05, 0.5)

        # Adjust thresholds based on precision-recall balance
        # Return DELTA values that will be added to current thresholds
        precision_consensus = metrics.get('consensus').get('precision', 0.5)
        recall_consensus = metrics.get('consensus').get('recall', 0.5)

        # If precision > recall, lower threshold to increase recall (negative delta)
        # If recall > precision, raise threshold to increase precision (positive delta)
        if precision_consensus > recall_consensus + 0.1:
            threshold_delta = -self.learning_rates['threshold']
        elif recall_consensus > precision_consensus + 0.1:
            threshold_delta = self.learning_rates['threshold']
        else:
            threshold_delta = 0.0

        print(f'   🧠 Policy Update: f1_consensus={f1_consensus:.4f}, f1_b1={f1_b1_score:.4f}, f1_b2={f1_b2_score:.4f}')
        print(f'   🧠 Weights: w1: {w1_current:.4f} → {w1_new:.4f}')
        print(f'   🧠 Contamination: {contamination_current:.4f} → {contamination_new:.4f} (Δ={delta_contamination:.4f})')
        print(f'   🧠 Threshold deltas: consensus: {threshold_delta:.4f}, b1: {threshold_delta*0.5:.4f}, b2: {threshold_delta*0.5:.4f}')

        self.policy_params['w1'] = w1_new
        self.policy_params['w2'] = w2_new
        self.policy_params['contamination_rate'] = contamination_new
        self.policy_params['threshold_delta_consensus'] = threshold_delta
        self.policy_params['threshold_delta_b1'] = threshold_delta * 0.5
        self.policy_params['threshold_delta_b2'] = threshold_delta * 0.5
        self.data_quantity += 1

        return {
            'status': 'updated',
            'w1': self.policy_params['w1'],
            'w2': self.policy_params['w2'],
            'contamination_rate': self.policy_params['contamination_rate'],
            'threshold_delta_consensus': self.policy_params['threshold_delta_consensus'],
            'threshold_delta_b1': self.policy_params['threshold_delta_b1'],
            'threshold_delta_b2': self.policy_params['threshold_delta_b2']
        }

    def execute(self, metrics: Dict) -> Dict[str, Any]:
        policy_update = self.update_policy_ppo(metrics)
        knowledge_graph = {
            'w1': float(self.policy_params['w1']),
            'w2': float(self.policy_params['w2']),
            'contamination_rate': float(self.policy_params['contamination_rate']),
            'threshold_delta_consensus': float(self.policy_params['threshold_delta_consensus']),
            'threshold_delta_b1': float(self.policy_params['threshold_delta_b1']),
            'threshold_delta_b2': float(self.policy_params['threshold_delta_b2']),
            'data_quantity': self.data_quantity
        }
        payload = {'timestamp': datetime.now().isoformat(), 'agent': self.name, 'policy': self.policy_params.copy(), 'knowledge_graph': knowledge_graph}
        self.publish(MQTT_CONFIG['topics']['policy_updates'], payload)
        self.publish(MQTT_CONFIG['topics']['knowledge_graph'], knowledge_graph)
        return payload

class CloudAgentE(MQTTAgent):
    def __init__(self, name='E_MetaAgent_SHAP', broker=None, background_data: np.ndarray = None):
        super().__init__(name, broker or message_broker)
        self.audit_log = []

    def execute(self, predictions: np.ndarray, scores: np.ndarray, y_true: np.ndarray, iteration: int) -> Dict[str, Any]:
        pred_binary = (predictions > 0).astype(int)
        metrics = {
            'accuracy': float(accuracy_score(y_true, pred_binary)),
            'precision': float(precision_score(y_true, pred_binary, zero_division=0)),
            'recall': float(recall_score(y_true, pred_binary, zero_division=0)),
            'f1': float(f1_score(y_true, pred_binary, zero_division=0)),
            'roc_auc': float(roc_auc_score(y_true, scores) if len(np.unique(y_true)) > 1 else 0.5),
            'mcc': float(matthews_corrcoef(y_true, pred_binary))
        }
        audit_report = {'iteration': iteration, 'metrics': metrics, 'status': 'PASS' if metrics['f1'] > 0.6 else 'REVIEW_NEEDED'}
        self.audit_log.append(audit_report)
        payload = {'timestamp': datetime.now().isoformat(), 'agent': self.name, 'iteration': iteration, 'audit_metrics': metrics}
        self.publish(MQTT_CONFIG['topics']['monitoring_logs'], payload)
        return payload

print("✅ Cloud Agents D, E defined")

✅ Cloud Agents D, E defined


In [ ]:
class SEMAS_COMPLETE(MQTTAgent):
    def __init__(self, name='SEMAS_COMPLETE', broker: MessageBroker = None, background_data: np.ndarray = None):
        super().__init__(name, broker or message_broker)
        self.broker = broker or message_broker
        self.name = name
        self.edge_a1 = EdgeSubagentA1(broker=self.broker)
        self.edge_a2 = EdgeSubagentA2(broker=self.broker)
        self.fog_b = FogOrchestrationAgentB(broker=self.broker)
        self.fog_b1 = FogSubagentB1(broker=self.broker)
        self.fog_b2 = FogSubagentB2(broker=self.broker)
        self.fog_b2_rul = FogSubagentB2_RUL(broker=self.broker)
        self.fog_b3 = FogSubagentB3(broker=self.broker)
        self.fog_c = FogSubagentC(broker=self.broker, use_mock=True)
        self.cloud_d = CloudAgentD(broker=self.broker)
        self.cloud_e = CloudAgentE(broker=self.broker, background_data=background_data)
        self.execution_history = []
        # Store separate thresholds for consensus, B1, and B2
        self.threshold_consensus = None
        self.threshold_b1 = None
        self.threshold_b2 = None

    def train(self, X_train: np.ndarray, RUL_train: np.ndarray = None):
        print(f"Training {self.name} models...")
        X_train_data = X_train.values if hasattr(X_train, 'values') else X_train
        self.fog_b1.train(X_train_data)
        self.fog_b2.train(X_train_data)
        if RUL_train is not None:
            self.fog_b2_rul.train(X_train_data, RUL_train)
            print("✅ RUL model trained")
        print("✅ Training complete")

    def create_predictions(self, X_test_data: np.ndarray, scores: np.ndarray, y_test_true: np.ndarray = None,
                         is_first_iteration: bool = False, score_type: str = 'consensus',
                         policy_threshold: float = None, adaptive_mode: bool = True):
        rul_result = self.fog_b2_rul.execute(X_test_data)
        scores = np.array(scores)

        # Select the appropriate threshold based on score_type
        if score_type == 'consensus':
            threshold_attr = 'threshold_consensus'
        elif score_type == 'b1':
            threshold_attr = 'threshold_b1'
        elif score_type == 'b2':
            threshold_attr = 'threshold_b2'
        else:
            threshold_attr = 'threshold_consensus'

        current_threshold = getattr(self, threshold_attr)

        # In adaptive mode, use policy-provided threshold if available
        if adaptive_mode and policy_threshold is not None:
            new_threshold = policy_threshold
            setattr(self, threshold_attr, new_threshold)
            current_threshold = new_threshold
        # Otherwise, calculate threshold on first iteration or if None
        elif is_first_iteration or current_threshold is None:
            if y_test_true is not None:
                from sklearn.metrics import precision_recall_curve
                prec, rec, thresholds = precision_recall_curve(y_test_true, scores)
                f1_scores = 2 * (prec * rec) / (prec + rec + 1e-10)
                new_threshold = thresholds[np.argmax(f1_scores)] if len(thresholds) > 0 else np.percentile(scores, 75)
                setattr(self, threshold_attr, new_threshold)
                print(f'   🎯 [{score_type.upper()}] Threshold calibrated: {new_threshold:.4f}')
            else:
                new_threshold = np.percentile(scores, 75)
                setattr(self, threshold_attr, new_threshold)
                print(f'   🎯 [{score_type.upper()}] Threshold set to 75th percentile: {new_threshold:.4f}')

            current_threshold = new_threshold

        # Use the stored threshold for predictions
        adjusted_predictions = np.where(scores >= current_threshold, 1, 0)
        return adjusted_predictions, rul_result, scores


    def evaluation_metrics(self, y_test_true: np.ndarray, adjusted_predictions: np.ndarray, scores: np.ndarray):
        metrics = {
            'accuracy': float(accuracy_score(y_test_true, adjusted_predictions)),
            'precision': float(precision_score(y_test_true, adjusted_predictions, zero_division=0)),
            'recall': float(recall_score(y_test_true, adjusted_predictions, zero_division=0)),
            'f1': float(f1_score(y_test_true, adjusted_predictions, zero_division=0)),
            'roc_auc': float(roc_auc_score(y_test_true, scores) if len(np.unique(y_test_true)) > 1 else 0.5),
        }
        return metrics

    def execute(self, X_test: np.ndarray, y_test_true: np.ndarray, y_test_rul: np.ndarray = None, iteration: int = 1, adaptive_mode: bool = True) -> Dict[str, Any]:
        X_test_data = X_test.values if hasattr(X_test, 'values') else X_test
        exec_start = time.time()
        print(f'\nSEMAS Iteration {iteration}')

        # Get policy updates from knowledge graph (for iterations > 1)
        policy_thresholds = {}
        policy_contamination = None
        if iteration > 1 and adaptive_mode:
            kg_msg = self.broker.get_latest(MQTT_CONFIG['topics']['knowledge_graph'])
            if kg_msg:
                policy_thresholds['consensus'] = kg_msg.get('payload', {}).get('threshold_consensus')
                policy_thresholds['b1'] = kg_msg.get('payload', {}).get('threshold_b1')
                policy_thresholds['b2'] = kg_msg.get('payload', {}).get('threshold_b2')
                policy_contamination = kg_msg.get('payload', {}).get('contamination_rate')

                # Apply contamination update to B1 model
                if policy_contamination is not None:
                    self.fog_b1.update_contamination(policy_contamination)

        self.edge_a1.execute(X_test_data)
        self.edge_a2.execute(X_test_data)
        b1_result = self.fog_b1.execute(X_test_data)
        b2_result = self.fog_b2.execute(X_test_data)
        b3_result = self.fog_b3.execute()
        c_result = self.fog_c.execute()
        consensus_scores = None
        scores_b1_norm = None
        scores_b2_norm = None
        if b3_result:
            consensus_scores = np.array(b3_result['consensus_scores'])
            scores_b1_norm = np.array(b3_result['scores_b1_norm'])
            scores_b2_norm = np.array(b3_result['scores_b2_norm'])
            predict_exec_start_time = time.time()
            adjusted_predictions, rul_result, consensus_array = self.create_predictions(
                X_test_data, consensus_scores, y_test_true,
                is_first_iteration=(iteration == 1), score_type='consensus',
                policy_threshold=policy_thresholds.get('consensus'), adaptive_mode=adaptive_mode)
            predict_exec_end_time = time.time()
            predict_exec_time = predict_exec_end_time - predict_exec_start_time
            adjusted_predictions_b1, rul_result_b1, scores_b1_norm = self.create_predictions(
                X_test_data, scores_b1_norm, y_test_true,
                is_first_iteration=(iteration == 1), score_type='b1',
                policy_threshold=policy_thresholds.get('b1'), adaptive_mode=adaptive_mode)
            adjusted_predictions_b2, rul_result_b2, scores_b2_norm = self.create_predictions(
                X_test_data, scores_b2_norm, y_test_true,
                is_first_iteration=(iteration == 1), score_type='b2',
                policy_threshold=policy_thresholds.get('b2'), adaptive_mode=adaptive_mode)
        else:
            adjusted_predictions, rul_result = None, None
            adjusted_predictions_b1, rul_result_b1 = None, None
            adjusted_predictions_b2, rul_result_b2 = None, None
        if b3_result:
            self.fog_b.detect_drift(np.array(b3_result['consensus_scores']))
        metrics = {}
        if adjusted_predictions is not None:
            eval_exec_start_time = time.time()
            metrics_consensus = self.evaluation_metrics(y_test_true, adjusted_predictions, consensus_array)
            eval_exec_end_time = time.time()
            eval_exec_time = eval_exec_end_time - eval_exec_start_time
            metrics_consensus['eval_exec_time'] = eval_exec_time
            metrics_consensus['predict_exec_time'] = predict_exec_time
            metrics_b1 = self.evaluation_metrics(y_test_true, adjusted_predictions_b1, scores_b1_norm)
            metrics_b2 = self.evaluation_metrics(y_test_true, adjusted_predictions_b2, scores_b2_norm)
            if rul_result is not None and y_test_rul is not None:
                rul_preds = np.array(rul_result['rul_predictions'])
                y_test_rul_array = y_test_rul.values if hasattr(y_test_rul, 'values') else y_test_rul
                metrics_consensus['rul_mae'] = float(mean_absolute_error(y_test_rul_array[:len(rul_preds)], rul_preds))
                metrics_consensus['rul_rmse'] = float(np.sqrt(mean_squared_error(y_test_rul_array[:len(rul_preds)], rul_preds)))
            metrics['consensus'] = metrics_consensus
            metrics['scores_b1_norm'] = metrics_b1
            metrics['scores_b2_norm'] = metrics_b2
            d_result = self.cloud_d.execute(metrics)
            e_result = self.cloud_e.execute(adjusted_predictions, consensus_array, y_test_true, iteration)

        exec_end = time.time()
        execution_time_ms = (exec_end - exec_start) * 1000
        metrics['execution_time_ms'] = execution_time_ms

        # Store policy state and thresholds in execution result
        execution_result = {
            'iteration': iteration,
            'timestamp': datetime.now().isoformat(),
            'metrics': metrics,
            'metrics_b1': metrics_b1,
            'metrics_b2': metrics_b2,
            'agent': self.name,
            'execution_time_ms': execution_time_ms,
            'policy_state': {
                'contamination': policy_contamination if policy_contamination else self.fog_b1.contamination,
                'threshold_consensus': self.threshold_consensus,
                'threshold_b1': self.threshold_b1,
                'threshold_b2': self.threshold_b2
            }
        }
        self.publish(MQTT_CONFIG['topics']['monitoring_logs'], execution_result)

        self.execution_history.append(execution_result)
        return execution_result

print("✅ SEMAS COMPLETE System defined")

✅ SEMAS COMPLETE System defined


In [ ]:
def run_semas_complete(dataset_key: str, dataset_config: Dict[str, Any], num_iterations: int = 3):
    print(f'\n📊 Dataset: {dataset_config["name"]}')
    print('=' * 80)
    semas = SEMAS_COMPLETE(broker=message_broker, background_data=dataset_config['X_train'].values if hasattr(dataset_config['X_train'], 'values') else dataset_config['X_train'])
    semas.train(dataset_config['X_train'], dataset_config.get('y_test_rul'))
    X_test = dataset_config['X_test']
    y_test_binary = dataset_config['y_test_anomaly'].values if hasattr(dataset_config['y_test_anomaly'], 'values') else dataset_config['y_test_anomaly']
    y_test_rul = dataset_config.get('y_test_rul')
    all_iterations = []
    for iteration in range(num_iterations):
        result = semas.execute(X_test, y_test_binary, y_test_rul, iteration=iteration + 1)
        if result['metrics']:
            print(f'\n   📈 Iteration {iteration + 1} Metrics:')
            for (metric, metric_values) in result['metrics'].items():
                print('-----------')
                print(f'metric: {metric}')
                print(f'      metric_values: {metric_values}')
        all_iterations.append(result)
    return {'dataset': dataset_key, 'iterations': all_iterations, 'semas_instance': semas}

print("\n" + "="*80)
print('🚀 RUNNING SEMAS-COMPLETE ON BOILER DATASET')
print("="*80)
semas_boiler = run_semas_complete('boiler', CONFIG['datasets']['boiler'], num_iterations=3)

print("\n" + "="*80)
print('🚀 RUNNING SEMAS-COMPLETE ON WIND TURBINE DATASET (FIXED LABELS)')
print("="*80)
semas_turbine = run_semas_complete('wind_turbine', CONFIG['datasets']['wind_turbine'], num_iterations=3)



🚀 RUNNING SEMAS-COMPLETE ON BOILER DATASET

📊 Dataset: Boiler Emulator
Training SEMAS_COMPLETE models...
   ✅ B1 trained with contamination=0.3200
✅ RUL model trained
✅ Training complete

SEMAS Iteration 1
weights None
w1: 0.4; w2: 0.6
top 5 consensus_scores: [0.26434306 0.31967432 0.28382108 0.31092658 0.08892104]
   🎯 [CONSENSUS] Threshold calibrated: 0.2439
   🎯 [B1] Threshold calibrated: 0.6073
   🎯 [B2] Threshold calibrated: 0.0000
   🧠 Policy Update: f1_consensus=0.5341, f1_b1=0.5341, f1_b2=0.4883
   🧠 Weights: w1: 0.4000 → 0.4096
   🧠 Contamination: 0.3200 → 0.3400 (Δ=0.0200)
   🧠 Threshold deltas: consensus: 0.0500, b1: 0.0250, b2: 0.0250

   📈 Iteration 1 Metrics:
-----------
metric: consensus
      metric_values: {'accuracy': 0.530608504398827, 'precision': 0.3928284720363928, 'recall': 0.8340909090909091, 'f1': 0.5341095142805167, 'roc_auc': 0.6694860537190083, 'eval_exec_time': 0.01642465591430664, 'predict_exec_time': 1.4065380096435547, 'rul_mae': 31.32182502746582, 'rul

In [ ]:
print('\n' + '=' * 80)
print('📊 METRICS ANALYSIS')
print('=' * 80)

def analyze_results(semas_results, dataset_name):
    print(f'\n{dataset_name.upper()} DATASET')
    print('-' * 80)
    iterations = semas_results['iterations']
    all_metrics = []
    for i, iter_result in enumerate(iterations):
        if iter_result['metrics']['consensus']:
            metrics_copy = iter_result['metrics']['consensus'].copy()
            metrics_copy['iteration'] = i + 1
            all_metrics.append(metrics_copy)
    if all_metrics:
        metrics_df = pd.DataFrame(all_metrics)
        print(metrics_df.to_string(index=False))
        print(f'\nAverage F1: {metrics_df["f1"].mean():.4f}, Precision: {metrics_df["precision"].mean():.4f}, Recall: {metrics_df["recall"].mean():.4f}')

analyze_results(semas_boiler, 'boiler')
analyze_results(semas_turbine, 'wind_turbine')



📊 METRICS ANALYSIS

BOILER DATASET
--------------------------------------------------------------------------------
 accuracy  precision   recall       f1  roc_auc  eval_exec_time  predict_exec_time   rul_mae  rul_rmse  iteration
 0.530609   0.392828 0.834091 0.534110 0.669486        0.016425           1.406538 31.321825 37.601826          1
 0.515396   0.386550 0.855682 0.532532 0.669488        0.011083           0.736774 31.321825 37.601826          2
 0.499084   0.379190 0.867614 0.527735 0.669490        0.010833           1.186018 31.321825 37.601826          3

Average F1: 0.5315, Precision: 0.3862, Recall: 0.8525

WIND_TURBINE DATASET
--------------------------------------------------------------------------------
 accuracy  precision   recall       f1  roc_auc  eval_exec_time  predict_exec_time  rul_mae  rul_rmse  iteration
 0.924242   0.924242 1.000000 0.960630 0.537705        0.008093           0.680690 7.201471 13.091529          1
 0.909091   0.923077 0.983607 0.952381 0.54

In [ ]:
# ============================================================================
# PERFORMANCE RESULTS COLLECTION FOR CROSS-PIPELINE COMPARISON
# ============================================================================
import pickle
from datetime import datetime

def collect_performance_results(semas_results, dataset_name, system_name='SEMAS'):
    """
    Collect comprehensive performance metrics for comparison across pipelines
    Enhanced version with policy evolution tracking (w1/w2 weights)
    """
    iterations_data = semas_results['iterations']
    semas_instance = semas_results.get('semas_instance')

    # Extract metrics from each iteration
    performance_log = {
        'system': system_name,
        'dataset': dataset_name,
        'timestamp': datetime.now().isoformat(),
        'num_iterations': len(iterations_data),
        'iterations': []
    }

    for idx, iter_result in enumerate(iterations_data, 1):
        iteration_metrics = {
            'iteration': idx,
            'metrics': {},
            'latency_ms': iter_result.get('execution_time_ms', 0),
            'layer_times': {}
        }

        # Extract layer execution times if available
        if iter_result.get('metrics') and iter_result['metrics'].get('consensus'):
            consensus = iter_result['metrics']['consensus']
            iteration_metrics['layer_times'] = {
                'eval_time': consensus.get('eval_exec_time', 0) * 1000,  # Convert to ms
                'predict_time': consensus.get('predict_exec_time', 0) * 1000
            }

        # Extract consensus metrics
        if iter_result.get('metrics') and iter_result['metrics'].get('consensus'):
            consensus = iter_result['metrics']['consensus']
            iteration_metrics['metrics'] = {
                'accuracy': consensus.get('accuracy', 0),
                'precision': consensus.get('precision', 0),
                'recall': consensus.get('recall', 0),
                'f1': consensus.get('f1', 0),
                'roc_auc': consensus.get('roc_auc', 0),
                'mcc': consensus.get('mcc', 0),
                'rul_mae': consensus.get('rul_mae', 0),
                'rul_rmse': consensus.get('rul_rmse', 0)
            }

        performance_log['iterations'].append(iteration_metrics)

    # Calculate aggregated statistics
    if performance_log['iterations']:
        all_f1 = [it['metrics'].get('f1', 0) for it in performance_log['iterations']]
        all_precision = [it['metrics'].get('precision', 0) for it in performance_log['iterations']]
        all_recall = [it['metrics'].get('recall', 0) for it in performance_log['iterations']]
        all_latency = [it['latency_ms'] for it in performance_log['iterations']]

        performance_log['summary'] = {
            'avg_f1': np.mean(all_f1),
            'std_f1': np.std(all_f1),
            'avg_precision': np.mean(all_precision),
            'avg_recall': np.mean(all_recall),
            'avg_latency_ms': np.mean(all_latency),
            'f1_trajectory': all_f1,
            'f1_improvement': all_f1[-1] - all_f1[0] if len(all_f1) > 1 else 0
        }

        # Extract policy evolution (w1/w2 weights from CloudAgentD)
        if semas_instance and hasattr(semas_instance, 'cloud_d'):
            cloud_d = semas_instance.cloud_d

            # Get weight history from knowledge graph messages
            weight_history = []
            kg_messages = message_broker.get_all(MQTT_CONFIG['topics']['knowledge_graph'])

            for msg in kg_messages:
                if 'payload' in msg:
                    payload = msg['payload']
                    weight_history.append({
                        'w1': payload.get('w1', 0.4),
                        'w2': payload.get('w2', 0.6),
                        'data_quantity': payload.get('data_quantity', 0)
                    })

            if weight_history:
                performance_log['policy_evolution'] = {
                    'initial_w1': weight_history[0]['w1'],
                    'final_w1': weight_history[-1]['w1'],
                    'initial_w2': weight_history[0]['w2'],
                    'final_w2': weight_history[-1]['w2'],
                    'weight_changes': len(weight_history) - 1,
                    'weight_history': weight_history
                }

    return performance_log

# Collect performance results for both datasets
print('\n' + '='*80)
print('📦 COLLECTING SEMAS PERFORMANCE RESULTS')
print('='*80)

semas_performance_boiler = collect_performance_results(semas_boiler, 'boiler', 'SEMAS')
semas_performance_turbine = collect_performance_results(semas_turbine, 'wind_turbine', 'SEMAS')

print('\n✅ BOILER DATASET RESULTS:')
print(f'   Avg F1: {semas_performance_boiler["summary"]["avg_f1"]:.4f} ± {semas_performance_boiler["summary"]["std_f1"]:.4f}')
print(f'   Avg Precision: {semas_performance_boiler["summary"]["avg_precision"]:.4f}')
print(f'   Avg Recall: {semas_performance_boiler["summary"]["avg_recall"]:.4f}')
print(f'   Avg Latency: {semas_performance_boiler["summary"]["avg_latency_ms"]:.2f}ms')
print(f'   F1 Trajectory: {[f"{x:.4f}" for x in semas_performance_boiler["summary"]["f1_trajectory"]]}')
print(f'   F1 Improvement: {semas_performance_boiler["summary"]["f1_improvement"]:.4f}')

print('\n✅ WIND TURBINE DATASET RESULTS:')
print(f'   Avg F1: {semas_performance_turbine["summary"]["avg_f1"]:.4f} ± {semas_performance_turbine["summary"]["std_f1"]:.4f}')
print(f'   Avg Precision: {semas_performance_turbine["summary"]["avg_precision"]:.4f}')
print(f'   Avg Recall: {semas_performance_turbine["summary"]["avg_recall"]:.4f}')
print(f'   Avg Latency: {semas_performance_turbine["summary"]["avg_latency_ms"]:.2f}ms')
print(f'   F1 Trajectory: {[f"{x:.4f}" for x in semas_performance_turbine["summary"]["f1_trajectory"]]}')
print(f'   F1 Improvement: {semas_performance_turbine["summary"]["f1_improvement"]:.4f}')

# Save results to pickle files for cross-pipeline comparison
try:
    with open('semas_performance_boiler.pkl', 'wb') as f:
        pickle.dump(semas_performance_boiler, f)
    with open('semas_performance_turbine.pkl', 'wb') as f:
        pickle.dump(semas_performance_turbine, f)
    print('\n💾 Results saved to: semas_performance_boiler.pkl, semas_performance_turbine.pkl')
except Exception as e:
    print(f'\n⚠️  Could not save pickle files: {e}')

print('\n' + '='*80)
print('✅ PERFORMANCE COLLECTION COMPLETE')
print('='*80)


📦 COLLECTING SEMAS PERFORMANCE RESULTS

✅ BOILER DATASET RESULTS:
   Avg F1: 0.5315 ± 0.0027
   Avg Precision: 0.3862
   Avg Recall: 0.8525
   Avg Latency: 4432.58ms
   F1 Trajectory: ['0.5341', '0.5325', '0.5277']
   F1 Improvement: -0.0064

✅ WIND TURBINE DATASET RESULTS:
   Avg F1: 0.9551 ± 0.0039
   Avg Precision: 0.9235
   Avg Recall: 0.9891
   Avg Latency: 1003.96ms
   F1 Trajectory: ['0.9606', '0.9524', '0.9524']
   F1 Improvement: -0.0082

💾 Results saved to: semas_performance_boiler.pkl, semas_performance_turbine.pkl

✅ PERFORMANCE COLLECTION COMPLETE
